In [5]:
import cv2
import numpy as np
from PIL import Image, ImageTk
import os
import tkinter as tk
from tkinter import ttk

IMG_PATH = {
    "img1": "img/picture1.png",
    "img2": "img/picture2.png",
    "img3": "img/picture3.jpg"
}
img_keys = list(IMG_PATH.keys())
current_idx = 0

In [14]:
class VisionGuiApp:
    def __init__(self, root, image_dict, keys):
        self.root = root
        self.root.title("LAB HSV - Centered Layout")
        self.root.geometry("1450x750")
        self.root.configure(bg="white")
        
        self.image_dict = image_dict
        self.keys = keys
        self.current_idx = 0

        tk.Label(root, text="Name Picture", font=("Arial Bold", 32), bg="white").pack(pady=20)

        self.middle_frame = tk.Frame(root, bg="white")
        self.middle_frame.pack(expand=True, fill="both")
        
        self.middle_frame.grid_columnconfigure(0, weight=1) 
        self.middle_frame.grid_columnconfigure(6, weight=1) 

        # ปุ่ม PREV
        self.btn_back = tk.Button(self.middle_frame, text="◀ PREV", font=("Arial Bold", 12), 
                                  bg="#88a404", fg="white", width=10, height=2, command=self.prev_image)
        self.btn_back.grid(row=0, column=1, padx=20)

        # Original Image
        self.orig_container = tk.Frame(self.middle_frame, bg="white")
        self.orig_container.grid(row=0, column=2, padx=10)
        tk.Label(self.orig_container, text="Original Image", font=("Arial Bold", 14), bg="white").pack()
        self.label_orig = tk.Label(self.orig_container, bg="#f0f0f0", relief="solid", borderwidth=1)
        self.label_orig.pack(pady=5)

        # Mask Preview
        self.mask_container = tk.Frame(self.middle_frame, bg="white")
        self.mask_container.grid(row=0, column=3, padx=10)
        tk.Label(self.mask_container, text="Color preview", font=("Arial Bold", 14), bg="white", fg="red").pack()
        self.label_mask = tk.Label(self.mask_container, bg="#f0f0f0", relief="solid", borderwidth=1)
        self.label_mask.pack(pady=5)

        # Result Image
        self.res_container = tk.Frame(self.middle_frame, bg="white")
        self.res_container.grid(row=0, column=4, padx=10)
        tk.Label(self.res_container, text="Result Picture", font=("Arial Bold", 14), bg="white").pack()
        self.label_res = tk.Label(self.res_container, bg="#f0f0f0", relief="solid", borderwidth=1)
        self.label_res.pack(pady=5)

        # button NEXT
        self.btn_next = tk.Button(self.middle_frame, text="NEXT ▶", font=("Arial Bold", 12), 
                                  bg="#88a404", fg="white", width=10, height=2, command=self.next_image)
        self.btn_next.grid(row=0, column=5, padx=20)

        self.bottom_container = tk.Frame(root, bg="white")
        self.bottom_container.pack(side="bottom", fill="x", pady=20)
        
        self.control_frame = tk.Frame(self.bottom_container, bg="white")
        self.control_frame.pack(anchor="center")

        self.lower_sliders = self.create_hsv_group(self.control_frame, "lower color", 0)
        self.upper_sliders = self.create_hsv_group(self.control_frame, "Upper color", 255)

        self.load_and_process()

    def create_hsv_group(self, parent, title, default_val):

        frame = tk.Frame(parent, bg="white", highlightbackground="#ddd", highlightthickness=1)
        frame.pack(side="left", padx=20, pady=10)
        
        tk.Label(frame, text=title, font=("Arial Bold", 16), fg="red", bg="white").pack(pady=5)
        
        sliders = {}
        for name in ['H', 'S', 'V']:
            row = tk.Frame(frame, bg="white")
            row.pack(fill="x", padx=10)
            tk.Label(row, text=name, font=("Arial Bold", 12), width=2, bg="white").pack(side="left")
            max_val = 179 if name == 'H' else 255
            s = tk.Scale(row, from_=0, to=max_val, orient="horizontal", bg="white", length=250,
                         highlightthickness=0, command=lambda x: self.load_and_process())
            s.set(default_val)
            s.pack(side="left", expand=True, fill="x", padx=5)
            sliders[name] = s
        return sliders

    def load_and_process(self):
        path = self.image_dict[self.keys[self.current_idx]]
        img = cv2.imread(path)
        if img is None: return

        # ดึงค่า HSV จาก Slider
        l_hsv_vals = [self.lower_sliders['H'].get(), self.lower_sliders['S'].get(), self.lower_sliders['V'].get()]
        u_hsv_vals = [self.upper_sliders['H'].get(), self.upper_sliders['S'].get(), self.upper_sliders['V'].get()]

        l_hsv = np.array(l_hsv_vals)
        u_hsv = np.array(u_hsv_vals)

        # 1. ประมวลผลภาพปกติ
        hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv_img, l_hsv, u_hsv)
        res = cv2.bitwise_and(img, img, mask=mask)

        # 2. สร้างภาพ Color Preview ขนาดเล็ก (เช่น 150x150 พิกเซล)
        # สร้าง Array เปล่าในระบบ HSV
        preview_size = 150
        color_img = np.zeros((preview_size, preview_size, 3), dtype=np.uint8)
        
        # เติมสีฝั่งซ้าย (Lower) และขวา (Upper)
        color_img[:, :75] = l_hsv_vals  # ครึ่งซ้าย
        color_img[:, 75:] = u_hsv_vals  # ครึ่งขวา
        
        # แปลง HSV เป็น BGR เพื่อแสดงผลสีที่ถูกต้อง
        color_preview_bgr = cv2.cvtColor(color_img, cv2.COLOR_HSV2BGR)

        # 3. อัปเดตการแสดงผล
        self.update_label(img, self.label_orig, (380, 280))     # ขนาดปกติ
        self.update_label(color_preview_bgr, self.label_mask, (150, 150)) # ขนาดเล็ก (Fill สี)
        self.update_label(res, self.label_res, (380, 280))      # ขนาดปกติ

    def update_label(self, cv_img, label, size):
        # แปลงเป็น RGB สำหรับ Pillow
        rgb = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)
        pil_img = Image.fromarray(rgb)
        
        # ปรับขนาดตามที่ส่งมาใน parameter 'size'
        pil_img = pil_img.resize(size, Image.LANCZOS)
        tk_img = ImageTk.PhotoImage(pil_img)
        
        label.config(image=tk_img)
        label.image = tk_img

    def next_image(self):
        self.current_idx = (self.current_idx + 1) % len(self.keys)
        self.load_and_process()

    def prev_image(self):
        self.current_idx = (self.current_idx - 1) % len(self.keys)
        self.load_and_process()

In [17]:
if __name__ == "__main__":
    if not os.path.exists("img"):
        print("ERROR NOT FOUND img FOLDER")

    root = tk.Tk()
    app = VisionGuiApp(root, IMG_PATH, img_keys)
    root.mainloop()